In [544]:
from google.colab import drive
# Mount google drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Lab/ODoH-ML/missouri-third-round-24nov/2/
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Lab/ODoH-ML/missouri-third-round-24nov/2
final2.csv                  MEREGED_RAW_FINAL.csv
final3_cleaned.csv          MEREGED_RAW_FINAL_GROUPED.csv
final3_cleaned_grouped.csv  missouri_third_round_part_22_nov_24.pcap
final3.csv                  target_proxy.csv
final4.csv                  target_resolver.csv
final.csv                   test4_target_proxy.pcap


In [545]:
# we don't need proxy pcap though, we can use juse test pcap since it has packets from proxy too.

In [546]:
import pandas as pd

# Load dataset file for Target - PROXY
df_raw = pd.read_csv("target_proxy.csv",  encoding = "ISO-8859-1")
df_raw.columns

Index(['No.', 'Time', 'Source', 'Destination', 'Protocol', 'User-Agent',
       'Content-Type', 'HTTP Response in frame', 'HTTP Request in frame',
       'Time since first frame in this TCP stream', 'Name',
       'HTTP-Content-Length', 'TCP Segment Len', 'DNS Response In',
       'DNS Request In', 'Time since HTTP request', 'DNS Length', 'Info'],
      dtype='object')

In [547]:
df_raw.head()

,No.,Time,Source,Destination,Protocol,User-Agent,Content-Type,HTTP Response in frame,HTTP Request in frame,Time since first frame in this TCP stream,Name,HTTP-Content-Length,TCP Segment Len,DNS Response In,DNS Request In,Time since HTTP request,DNS Length,Info
0,1,0.000000,192.122.236.103,204.102.244.58,TCP,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,48352 > 2223 [SYN] Seq=0 Win=32767 Len=0
1,2,0.000045,204.102.244.58,192.122.236.103,TCP,NaN,NaN,NaN,NaN,0.000045,NaN,NaN,0,NaN,NaN,NaN,NaN,"2223 > 48352 [SYN, ACK] Seq=0 Ack=1 Win=3276..."
2,3,0.000029,192.122.236.103,204.102.244.58,TCP,NaN,NaN,NaN,NaN,0.000074,NaN,NaN,0,NaN,NaN,NaN,NaN,48352 > 2223 [ACK] Seq=1 Ack=1 Win=32767 Len=0
3,4,0.000029,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,NaN,NaN,0.000103,NaN,173.0,353,NaN,NaN,NaN,NaN,POST /dns-query HTTP/1.1 (application/oblivio...
4,5,0.000028,204.102.244.58,192.122.236.103,TCP,NaN,NaN,NaN,NaN,0.000131,NaN,NaN,0,NaN,NaN,NaN,NaN,2223 > 48352 [ACK] Seq=1 Ack=354 Win=32767 L...


In [548]:
df_raw.Protocol.unique()

array(['TCP', 'HTTP'], dtype=object)

In [549]:
proxy_ip = '192.122.236.103' 
server_ip = '204.102.244.58'

In [550]:
df_http_target_to_proxy = df_raw[(df_raw.Protocol == "HTTP") & (df_raw.Source == server_ip) & (df_raw.Destination == proxy_ip)]
df_http_proxy_to_target = df_raw[(df_raw.Protocol == "HTTP") & (df_raw.Source == proxy_ip) & (df_raw.Destination == server_ip)]

In [551]:
df_http_target_to_proxy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 13 to 5192
Data columns (total 18 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   No.                                        1277 non-null   int64  
 1   Time                                       1277 non-null   float64
 2   Source                                     1277 non-null   object 
 3   Destination                                1277 non-null   object 
 4   Protocol                                   1277 non-null   object 
 5   User-Agent                                 1277 non-null   object 
 6   Content-Type                               1277 non-null   object 
 7   HTTP Response in frame                     0 non-null      float64
 8   HTTP Request in frame                      1277 non-null   float64
 9   Time since first frame in this TCP stream  1277 non-null   float64
 10  Name                   

In [552]:
df_http_proxy_to_target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1287 entries, 3 to 5188
Data columns (total 18 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   No.                                        1287 non-null   int64  
 1   Time                                       1287 non-null   float64
 2   Source                                     1287 non-null   object 
 3   Destination                                1287 non-null   object 
 4   Protocol                                   1287 non-null   object 
 5   User-Agent                                 1287 non-null   object 
 6   Content-Type                               1287 non-null   object 
 7   HTTP Response in frame                     1277 non-null   float64
 8   HTTP Request in frame                      0 non-null      float64
 9   Time since first frame in this TCP stream  1287 non-null   float64
 10  Name                    

In [553]:
df_http_target_to_proxy.iloc[1]["Source"]

'204.102.244.58'

Dropping useless columns

In [554]:
df_http_target_to_proxy_updated = df_http_target_to_proxy.drop(['Info', 'DNS Length', 'Name', 'DNS Response In', 'DNS Request In', 'HTTP Response in frame'], axis=1)

In [555]:
df_http_target_to_proxy_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 13 to 5192
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   No.                                        1277 non-null   int64  
 1   Time                                       1277 non-null   float64
 2   Source                                     1277 non-null   object 
 3   Destination                                1277 non-null   object 
 4   Protocol                                   1277 non-null   object 
 5   User-Agent                                 1277 non-null   object 
 6   Content-Type                               1277 non-null   object 
 7   HTTP Request in frame                      1277 non-null   float64
 8   Time since first frame in this TCP stream  1277 non-null   float64
 9   HTTP-Content-Length                        1277 non-null   float64
 10  TCP Segment Len        

In [556]:
queries = []

for i in range(len(df_http_target_to_proxy_updated)):
  query = df_http_target_to_proxy_updated.iloc[i]["User-Agent"]
  # print(query)
  x = query.find("QUESTION SECTION:")
  y = len("QUESTION SECTION:")
  z = query.find(";; ADDITIONAL SECTION")
  q = query[x+y+1:z]
  queries.append(q)

df_http_target_to_proxy_updated['Domain'] = queries
# queries

In [557]:
df_http_target_to_proxy_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 13 to 5192
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   No.                                        1277 non-null   int64  
 1   Time                                       1277 non-null   float64
 2   Source                                     1277 non-null   object 
 3   Destination                                1277 non-null   object 
 4   Protocol                                   1277 non-null   object 
 5   User-Agent                                 1277 non-null   object 
 6   Content-Type                               1277 non-null   object 
 7   HTTP Request in frame                      1277 non-null   float64
 8   Time since first frame in this TCP stream  1277 non-null   float64
 9   HTTP-Content-Length                        1277 non-null   float64
 10  TCP Segment Len        

In [558]:
df_http_target_to_proxy_updated2 = df_http_target_to_proxy_updated.rename({'Time': 'Target to Proxy - Time since previous frame in this TCP stream', 
                    'No.' : 'No. Target to Proxy', 
                    'User-Agent' : 'complete query',
                    'HTTP-Content-Length' : 'Target to Proxy HTTP-Content-Length',
                    'Time since first frame in this TCP stream' : 'Target to Proxy Time since first frame in this TCP stream'}, axis=1) 
df_http_target_to_proxy_updated2 = df_http_target_to_proxy_updated2.assign(Direction = 'TargetToProxy')

In [559]:
df_http_target_to_proxy_updated2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 13 to 5192
Data columns (total 14 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   No. Target to Proxy                                             1277 non-null   int64  
 1   Target to Proxy - Time since previous frame in this TCP stream  1277 non-null   float64
 2   Source                                                          1277 non-null   object 
 3   Destination                                                     1277 non-null   object 
 4   Protocol                                                        1277 non-null   object 
 5   complete query                                                  1277 non-null   object 
 6   Content-Type                                                    1277 non-null   object 
 7   HTTP Request in frame                             

In [560]:
df_http_proxy_to_target_updated = df_http_proxy_to_target.drop(['DNS Length', 'Time since HTTP request', 'Info', 'Name', 'DNS Response In', 'DNS Request In', 'HTTP Request in frame'], axis=1)

In [561]:
df_http_proxy_to_target_updated2 = df_http_proxy_to_target_updated.rename({'Time': 'Proxy to Target Time since previous frame in this TCP stream', 
                    'User-Agent' : 'Proxy to Target Encrypted Query',
                    'No.' : 'No. Proxy to Target',
                    'HTTP-Content-Length' : 'Proxy to Target HTTP-Content-Length',
                    'Time since first frame in this TCP stream' : 'Proxy to Target Time since first frame in this TCP stream'}, axis=1)  # new method
df_http_proxy_to_target_updated2 = df_http_proxy_to_target_updated2.assign(Direction = 'ProxyToTarget')

In [562]:
len(df_http_proxy_to_target_updated2)

1287

In [563]:
len(df_http_target_to_proxy_updated2)

1277

In [564]:
# dfxx = df_http_proxy_to_target_updated2[df_http_proxy_to_target_updated2.index.notnull()]
df_http_proxy_to_target_updated2 = df_http_proxy_to_target_updated2.dropna()
# df2=df.dropna(axis=0)

# Reset index after drop
df_http_proxy_to_target_updated2 = df_http_proxy_to_target_updated2.dropna().reset_index(drop=True)
len(df_http_proxy_to_target_updated2)

1277

In [565]:
list1 = df_http_proxy_to_target_updated2['HTTP Response in frame'].tolist()
list1
list2 = [int(a) for a in list1]
df_http_proxy_to_target_updated2['No. Target to Proxy'] = list2

In [566]:
list1

[14.0,
 18.0,
 22.0,
 26.0,
 30.0,
 41.0,
 39.0,
 49.0,
 47.0,
 55.0,
 57.0,
 65.0,
 63.0,
 71.0,
 73.0,
 79.0,
 81.0,
 89.0,
 87.0,
 97.0,
 95.0,
 103.0,
 105.0,
 111.0,
 113.0,
 119.0,
 121.0,
 129.0,
 127.0,
 137.0,
 135.0,
 143.0,
 145.0,
 151.0,
 153.0,
 159.0,
 161.0,
 169.0,
 167.0,
 175.0,
 177.0,
 183.0,
 185.0,
 191.0,
 193.0,
 201.0,
 199.0,
 207.0,
 209.0,
 217.0,
 215.0,
 223.0,
 225.0,
 233.0,
 231.0,
 239.0,
 241.0,
 249.0,
 247.0,
 255.0,
 257.0,
 265.0,
 263.0,
 271.0,
 273.0,
 281.0,
 279.0,
 289.0,
 287.0,
 295.0,
 297.0,
 303.0,
 305.0,
 313.0,
 311.0,
 319.0,
 321.0,
 327.0,
 329.0,
 337.0,
 335.0,
 343.0,
 345.0,
 351.0,
 353.0,
 361.0,
 359.0,
 367.0,
 369.0,
 377.0,
 375.0,
 383.0,
 385.0,
 393.0,
 391.0,
 399.0,
 401.0,
 407.0,
 409.0,
 417.0,
 415.0,
 423.0,
 425.0,
 431.0,
 433.0,
 439.0,
 441.0,
 447.0,
 449.0,
 455.0,
 457.0,
 465.0,
 463.0,
 471.0,
 473.0,
 479.0,
 481.0,
 487.0,
 489.0,
 495.0,
 497.0,
 503.0,
 505.0,
 513.0,
 511.0,
 519.0,
 521.0,
 527.

In [567]:
df_http_proxy_to_target_updated2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277 entries, 0 to 1276
Data columns (total 13 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   No. Proxy to Target                                           1277 non-null   int64  
 1   Proxy to Target Time since previous frame in this TCP stream  1277 non-null   float64
 2   Source                                                        1277 non-null   object 
 3   Destination                                                   1277 non-null   object 
 4   Protocol                                                      1277 non-null   object 
 5   Proxy to Target Encrypted Query                               1277 non-null   object 
 6   Content-Type                                                  1277 non-null   object 
 7   HTTP Response in frame                                        1277 no

In [568]:
df_http_target_to_proxy_updated2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 13 to 5192
Data columns (total 14 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   No. Target to Proxy                                             1277 non-null   int64  
 1   Target to Proxy - Time since previous frame in this TCP stream  1277 non-null   float64
 2   Source                                                          1277 non-null   object 
 3   Destination                                                     1277 non-null   object 
 4   Protocol                                                        1277 non-null   object 
 5   complete query                                                  1277 non-null   object 
 6   Content-Type                                                    1277 non-null   object 
 7   HTTP Request in frame                             

In [569]:
import pandas as pd
df_final = pd.merge(df_http_proxy_to_target_updated2, df_http_target_to_proxy_updated2, on='No. Target to Proxy', how='outer')
df_final.head()

,No. Proxy to Target,Proxy to Target Time since previous frame in this TCP stream,Source_x,Destination_x,Protocol_x,Proxy to Target Encrypted Query,Content-Type_x,HTTP Response in frame,Proxy to Target Time since first frame in this TCP stream,Proxy to Target HTTP-Content-Length,...,Protocol_y,complete query,Content-Type_y,HTTP Request in frame,Target to Proxy Time since first frame in this TCP stream,Target to Proxy HTTP-Content-Length,TCP Segment Len_y,Time since HTTP request,Domain,Direction_y
0,9,0.000029,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,14.0,0.000098,173.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 25325 ;...",application/oblivious-dns-message,9.0,0.376610,280.0,698,0.376512,;client-mo.test3.ch-geni-net.instageni.rnet.mi...,TargetToProxy
1,16,4.233926,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,18.0,4.610582,173.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 25325 ;...",application/oblivious-dns-message,16.0,4.725139,280.0,698,0.114557,;client-mo.test3.ch-geni-net.instageni.rnet.mi...,TargetToProxy
2,20,0.103085,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,22.0,4.828271,173.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 25866 ;...",application/oblivious-dns-message,20.0,5.008317,368.0,789,0.180046,;client-mo.test3.ch-geni-net.instageni.rnet.mi...,TargetToProxy
3,24,0.177221,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,26.0,5.185591,135.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 18980 ;...",application/oblivious-dns-message,24.0,5.230290,170.0,550,0.044699,;www.wikimedia.org.\\tIN\\t A,TargetToProxy
4,28,0.069576,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,30.0,5.299912,135.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 48696 ;...",application/oblivious-dns-message,28.0,5.449468,182.0,565,0.149556,;www.wikimedia.org.\\tIN\\t AAAA,TargetToProxy


In [570]:
# saving the dataframe
df_final.to_csv('final.csv')

In [571]:
df_final.iloc[2]['Domain']

';client-mo.test3.ch-geni-net.instageni.rnet.missouri.edu.\\\\tIN\\\\t AAAA  '

In [572]:
# groups_list = []
# # first or second query may belong to gini, we can select from next query which is for first website like google here
# tag = ""
# for i in range(len(df_final)):
#   if("client-wisconsin" in df_final.iloc[i]['Domain']):
#     print("this is for geni and we don't select it: ", df_final.iloc[i]['Domain'])
#   else:
#     tag = df_final.iloc[i]['Domain']
#     break


# for i in range(len(df_final) - 1):
#   if((df_final.iloc[i+1]['Proxy to Target Time since previous frame in this TCP stream'] - df_final.iloc[i]['Proxy to Target Time since previous frame in this TCP stream']) < 30):
#     groups_list.append(tag)
#   else:
#     tag = df_final.iloc[i+1]['Domain']
#     groups_list.append(tag)

# len(groups_list)

In [573]:
%ls

final2.csv                  MEREGED_RAW_FINAL.csv
final3_cleaned.csv          MEREGED_RAW_FINAL_GROUPED.csv
final3_cleaned_grouped.csv  missouri_third_round_part_22_nov_24.pcap
final3.csv                  target_proxy.csv
final4.csv                  target_resolver.csv
final.csv                   test4_target_proxy.pcap


In [574]:
import pandas as pd

# Load dataset file
df_resolver = pd.read_csv("target_resolver.csv",  encoding = "ISO-8859-1")
df_resolver.columns

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['No.', 'Time', 'Source', 'Destination', 'Protocol', 'User-Agent',
       'Content-Type', 'HTTP Response in frame', 'HTTP Request in frame',
       'Time since first frame in this TCP stream', 'Name',
       'HTTP-Content-Length', 'TCP Segment Len', 'DNS Response In',
       'DNS Request In', 'Time since HTTP request', 'DNS Length', 'Info'],
      dtype='object')

In [575]:
df_resolver.head(2)

,No.,Time,Source,Destination,Protocol,User-Agent,Content-Type,HTTP Response in frame,HTTP Request in frame,Time since first frame in this TCP stream,Name,HTTP-Content-Length,TCP Segment Len,DNS Response In,DNS Request In,Time since HTTP request,DNS Length,Info
0,1,0.000000,204.102.244.58,165.134.212.49,SSH,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,44.0,NaN,NaN,NaN,NaN,Server: Encrypted packet (len=44)
1,2,0.000603,204.102.244.58,165.134.212.49,SSH,NaN,NaN,NaN,NaN,0.000603,NaN,NaN,108.0,NaN,NaN,NaN,NaN,Server: Encrypted packet (len=108)


In [576]:
dns_resolver_list = ['1.1.1.1','8.8.8.8','9.9.9.9']
df_target_to_resolver_updated = df_resolver[(df_resolver.Protocol == "DNS") & (df_resolver.Source == server_ip) & (df_resolver.Destination.isin(dns_resolver_list))]
df_resolver_to_target_updated = df_resolver[(df_resolver.Protocol == "DNS") & (df_resolver.Destination == server_ip) & (df_resolver.Source.isin(dns_resolver_list))]
df_resolver_to_target_updated.head()

,No.,Time,Source,Destination,Protocol,User-Agent,Content-Type,HTTP Response in frame,HTTP Request in frame,Time since first frame in this TCP stream,Name,HTTP-Content-Length,TCP Segment Len,DNS Response In,DNS Request In,Time since HTTP request,DNS Length,Info
280,281,0.011421,9.9.9.9,204.102.244.58,DNS,NaN,NaN,NaN,NaN,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,NaN,124.0,NaN,245.0,NaN,122.0,Standard query response 0x62ed A client-mo.tes...
304,305,0.022652,8.8.8.8,204.102.244.58,DNS,NaN,NaN,NaN,NaN,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,NaN,124.0,NaN,301.0,NaN,122.0,Standard query response 0x62ed A client-mo.tes...
320,321,0.100845,1.1.1.1,204.102.244.58,DNS,NaN,NaN,NaN,NaN,0.176711,client-mo.test3.ch-geni-net.instageni.rnet.mis...,NaN,161.0,NaN,318.0,NaN,159.0,Standard query response 0x650a AAAA client-mo....
335,336,0.038791,8.8.8.8,204.102.244.58,DNS,NaN,NaN,NaN,NaN,0.040640,www.wikimedia.org,NaN,83.0,NaN,334.0,NaN,81.0,Standard query response 0x4a24 A www.wikimedia...
388,389,0.022088,9.9.9.9,204.102.244.58,DNS,NaN,NaN,NaN,NaN,0.145537,www.wikimedia.org,NaN,95.0,NaN,362.0,NaN,93.0,Standard query response 0xbe38 AAAA www.wikime...


In [577]:
df_target_to_resolver_updated2 = df_target_to_resolver_updated.drop(['Time since HTTP request', 'HTTP Request in frame', 'DNS Request In', 'User-Agent', 'Content-Type', 'HTTP-Content-Length', 'HTTP Response in frame'], axis=1)
df_resolver_to_target_updated2 = df_resolver_to_target_updated.drop(['Time since HTTP request', 'HTTP Request in frame', 'DNS Response In', 'User-Agent', 'Content-Type', 'HTTP-Content-Length', 'HTTP Response in frame'], axis=1)

df_target_to_resolver_updated2 = df_target_to_resolver_updated2.rename({'Time since first frame in this TCP stream': 'Target to Resolver Time since first frame in this TCP stream',
                            }, axis=1) 

df_resolver_to_target_updated2 = df_resolver_to_target_updated2.rename({'Time since first frame in this TCP stream': 'Resolver to Target Time since first frame in this TCP stream',
                            }, axis=1) 

df_target_to_resolver_updated2 = df_target_to_resolver_updated2.rename({'DNS Response In' : 'Match DNS Response In'}, axis=1 )  

In [578]:
df_target_to_resolver_updated2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1287 entries, 241 to 30645
Data columns (total 11 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   No.                                                           1287 non-null   int64  
 1   Time                                                          1287 non-null   float64
 2   Source                                                        1287 non-null   object 
 3   Destination                                                   1287 non-null   object 
 4   Protocol                                                      1287 non-null   object 
 5   Target to Resolver Time since first frame in this TCP stream  1287 non-null   float64
 6   Name                                                          1287 non-null   object 
 7   TCP Segment Len                                               1287

In [579]:
df_resolver_to_target_updated2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 280 to 30651
Data columns (total 11 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   No.                                                           1277 non-null   int64  
 1   Time                                                          1277 non-null   float64
 2   Source                                                        1277 non-null   object 
 3   Destination                                                   1277 non-null   object 
 4   Protocol                                                      1277 non-null   object 
 5   Resolver to Target Time since first frame in this TCP stream  1277 non-null   float64
 6   Name                                                          1277 non-null   object 
 7   TCP Segment Len                                               1277

In [580]:
# dfxx = df_http_proxy_to_target_updated2[df_http_proxy_to_target_updated2.index.notnull()]
df_target_to_resolver_updated2 = df_target_to_resolver_updated2.dropna()
# df2=df.dropna(axis=0)

# Reset index after drop
df_target_to_resolver_updated2 = df_target_to_resolver_updated2.dropna().reset_index(drop=True)
len(df_target_to_resolver_updated2)

1277

In [581]:
list_dns_req_res = df_target_to_resolver_updated2['Match DNS Response In'].tolist()
list_dns_req_res2 = [int(a) for a in list_dns_req_res]
df_resolver_to_target_updated2['Match DNS Response In'] = list_dns_req_res2

In [582]:
df_resolver_to_target_updated2

,No.,Time,Source,Destination,Protocol,Resolver to Target Time since first frame in this TCP stream,Name,TCP Segment Len,DNS Request In,DNS Length,Info,Match DNS Response In
280,281,0.011421,9.9.9.9,204.102.244.58,DNS,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,245.0,122.0,Standard query response 0x62ed A client-mo.tes...,281
304,305,0.022652,8.8.8.8,204.102.244.58,DNS,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,301.0,122.0,Standard query response 0x62ed A client-mo.tes...,305
320,321,0.100845,1.1.1.1,204.102.244.58,DNS,0.176711,client-mo.test3.ch-geni-net.instageni.rnet.mis...,161.0,318.0,159.0,Standard query response 0x650a AAAA client-mo....,321
335,336,0.038791,8.8.8.8,204.102.244.58,DNS,0.040640,www.wikimedia.org,83.0,334.0,81.0,Standard query response 0x4a24 A www.wikimedia...,336
388,389,0.022088,9.9.9.9,204.102.244.58,DNS,0.145537,www.wikimedia.org,95.0,362.0,93.0,Standard query response 0xbe38 AAAA www.wikime...,389
...,...,...,...,...,...,...,...,...,...,...,...,...
30590,30591,0.008889,8.8.8.8,204.102.244.58,DNS,0.011580,fonts.googleapis.com,79.0,30589.0,77.0,Standard query response 0x0866 AAAA fonts.goog...,30591
30618,30619,0.006576,8.8.8.8,204.102.244.58,DNS,0.008588,fonts.gstatic.com,76.0,30616.0,74.0,Standard query response 0x6ea0 AAAA fonts.gsta...,30622
30621,30622,0.001214,1.1.1.1,204.102.244.58,DNS,0.012067,fonts.gstatic.com,64.0,30613.0,62.0,Standard query response 0xad8b A fonts.gstatic...,30619
30648,30649,0.000936,1.1.1.1,204.102.244.58,DNS,0.003659,blogger.googleusercontent.com,117.0,30646.0,115.0,Standard query response 0xff10 AAAA blogger.go...,30652


In [583]:
df_final_resolver_target = pd.merge(df_target_to_resolver_updated2, df_resolver_to_target_updated2, on='Match DNS Response In', how='outer')
df_final_resolver_target

,No._x,Time_x,Source_x,Destination_x,Protocol_x,Target to Resolver Time since first frame in this TCP stream,Name_x,TCP Segment Len_x,Match DNS Response In,DNS Length_x,...,Time_y,Source_y,Destination_y,Protocol_y,Resolver to Target Time since first frame in this TCP stream,Name_y,TCP Segment Len_y,DNS Request In,DNS Length_y,Info_y
0,245,0.000528,204.102.244.58,9.9.9.9,DNS,0.006256,client-mo.test3.ch-geni-net.instageni.rnet.mis...,86.0,281.0,84.0,...,0.011421,9.9.9.9,204.102.244.58,DNS,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,245.0,122.0,Standard query response 0x62ed A client-mo.tes...
1,301,0.000322,204.102.244.58,8.8.8.8,DNS,0.001042,client-mo.test3.ch-geni-net.instageni.rnet.mis...,86.0,305.0,84.0,...,0.022652,8.8.8.8,204.102.244.58,DNS,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,301.0,122.0,Standard query response 0x62ed A client-mo.tes...
2,318,0.000568,204.102.244.58,1.1.1.1,DNS,0.001931,client-mo.test3.ch-geni-net.instageni.rnet.mis...,86.0,321.0,84.0,...,0.100845,1.1.1.1,204.102.244.58,DNS,0.176711,client-mo.test3.ch-geni-net.instageni.rnet.mis...,161.0,318.0,159.0,Standard query response 0x650a AAAA client-mo....
3,334,0.000353,204.102.244.58,8.8.8.8,DNS,0.001093,www.wikimedia.org,48.0,336.0,46.0,...,0.038791,8.8.8.8,204.102.244.58,DNS,0.040640,www.wikimedia.org,83.0,334.0,81.0,Standard query response 0x4a24 A www.wikimedia...
4,362,0.000318,204.102.244.58,9.9.9.9,DNS,0.002726,www.wikimedia.org,48.0,389.0,46.0,...,0.022088,9.9.9.9,204.102.244.58,DNS,0.145537,www.wikimedia.org,95.0,362.0,93.0,Standard query response 0xbe38 AAAA www.wikime...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,30589,0.000898,204.102.244.58,8.8.8.8,DNS,0.002018,fonts.googleapis.com,51.0,30591.0,49.0,...,0.008889,8.8.8.8,204.102.244.58,DNS,0.011580,fonts.googleapis.com,79.0,30589.0,77.0,Standard query response 0x0866 AAAA fonts.goog...
1273,30613,0.000468,204.102.244.58,1.1.1.1,DNS,0.001863,fonts.gstatic.com,48.0,30622.0,46.0,...,0.006576,8.8.8.8,204.102.244.58,DNS,0.008588,fonts.gstatic.com,76.0,30616.0,74.0,Standard query response 0x6ea0 AAAA fonts.gsta...
1274,30616,0.000740,204.102.244.58,8.8.8.8,DNS,0.001347,fonts.gstatic.com,48.0,30619.0,46.0,...,0.001214,1.1.1.1,204.102.244.58,DNS,0.012067,fonts.gstatic.com,64.0,30613.0,62.0,Standard query response 0xad8b A fonts.gstatic...
1275,30643,0.000552,204.102.244.58,9.9.9.9,DNS,0.002009,blogger.googleusercontent.com,60.0,30652.0,58.0,...,0.000936,1.1.1.1,204.102.244.58,DNS,0.003659,blogger.googleusercontent.com,117.0,30646.0,115.0,Standard query response 0xff10 AAAA blogger.go...


In [584]:
list4 = df_final['complete query'].tolist()
list5 = df_final_resolver_target['Name_x'].tolist()
len(list5)

1277

In [585]:
all_set = False
for i in range(len(list4)):
  # if(i == len(list5)):
  #   break
  if list5[i] in list4[i]:
      all_set = True
  else:
      all_set = False
all_set

True

In [586]:
df_final

,No. Proxy to Target,Proxy to Target Time since previous frame in this TCP stream,Source_x,Destination_x,Protocol_x,Proxy to Target Encrypted Query,Content-Type_x,HTTP Response in frame,Proxy to Target Time since first frame in this TCP stream,Proxy to Target HTTP-Content-Length,...,Protocol_y,complete query,Content-Type_y,HTTP Request in frame,Target to Proxy Time since first frame in this TCP stream,Target to Proxy HTTP-Content-Length,TCP Segment Len_y,Time since HTTP request,Domain,Direction_y
0,9,0.000029,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,14.0,0.000098,173.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 25325 ;...",application/oblivious-dns-message,9.0,0.376610,280.0,698,0.376512,;client-mo.test3.ch-geni-net.instageni.rnet.mi...,TargetToProxy
1,16,4.233926,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,18.0,4.610582,173.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 25325 ;...",application/oblivious-dns-message,16.0,4.725139,280.0,698,0.114557,;client-mo.test3.ch-geni-net.instageni.rnet.mi...,TargetToProxy
2,20,0.103085,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,22.0,4.828271,173.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 25866 ;...",application/oblivious-dns-message,20.0,5.008317,368.0,789,0.180046,;client-mo.test3.ch-geni-net.instageni.rnet.mi...,TargetToProxy
3,24,0.177221,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,26.0,5.185591,135.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 18980 ;...",application/oblivious-dns-message,24.0,5.230290,170.0,550,0.044699,;www.wikimedia.org.\\tIN\\t A,TargetToProxy
4,28,0.069576,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,30.0,5.299912,135.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 48696 ;...",application/oblivious-dns-message,28.0,5.449468,182.0,565,0.149556,;www.wikimedia.org.\\tIN\\t AAAA,TargetToProxy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,5173,0.001590,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,5177.0,3.566349,138.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 2150 ;;...",application/oblivious-dns-message,5173.0,3.583389,138.0,523,0.017040,;fonts.googleapis.com.\\tIN\\t AAAA,TargetToProxy
1273,5179,0.367031,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,5185.0,3.950462,135.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 44427 ;...",application/oblivious-dns-message,5179.0,3.967704,120.0,500,0.017242,;fonts.gstatic.com.\\tIN\\t A,TargetToProxy
1274,5181,0.001537,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,5183.0,886.614018,135.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 28320 ;...",application/oblivious-dns-message,5181.0,886.627802,132.0,515,0.013784,;fonts.gstatic.com.\\tIN\\t AAAA,TargetToProxy
1275,5187,0.344038,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,5193.0,4.311781,147.0,...,HTTP,";; opcode: QUERY, status: NOERROR, id: 26112 ;...",application/oblivious-dns-message,5187.0,4.329839,230.0,622,0.018058,;blogger.googleusercontent.com.\\tIN\\t A,TargetToProxy


In [587]:
df_final_resolver_target.head()

,No._x,Time_x,Source_x,Destination_x,Protocol_x,Target to Resolver Time since first frame in this TCP stream,Name_x,TCP Segment Len_x,Match DNS Response In,DNS Length_x,...,Time_y,Source_y,Destination_y,Protocol_y,Resolver to Target Time since first frame in this TCP stream,Name_y,TCP Segment Len_y,DNS Request In,DNS Length_y,Info_y
0,245,0.000528,204.102.244.58,9.9.9.9,DNS,0.006256,client-mo.test3.ch-geni-net.instageni.rnet.mis...,86.0,281.0,84.0,...,0.011421,9.9.9.9,204.102.244.58,DNS,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,245.0,122.0,Standard query response 0x62ed A client-mo.tes...
1,301,0.000322,204.102.244.58,8.8.8.8,DNS,0.001042,client-mo.test3.ch-geni-net.instageni.rnet.mis...,86.0,305.0,84.0,...,0.022652,8.8.8.8,204.102.244.58,DNS,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,301.0,122.0,Standard query response 0x62ed A client-mo.tes...
2,318,0.000568,204.102.244.58,1.1.1.1,DNS,0.001931,client-mo.test3.ch-geni-net.instageni.rnet.mis...,86.0,321.0,84.0,...,0.100845,1.1.1.1,204.102.244.58,DNS,0.176711,client-mo.test3.ch-geni-net.instageni.rnet.mis...,161.0,318.0,159.0,Standard query response 0x650a AAAA client-mo....
3,334,0.000353,204.102.244.58,8.8.8.8,DNS,0.001093,www.wikimedia.org,48.0,336.0,46.0,...,0.038791,8.8.8.8,204.102.244.58,DNS,0.040640,www.wikimedia.org,83.0,334.0,81.0,Standard query response 0x4a24 A www.wikimedia...
4,362,0.000318,204.102.244.58,9.9.9.9,DNS,0.002726,www.wikimedia.org,48.0,389.0,46.0,...,0.022088,9.9.9.9,204.102.244.58,DNS,0.145537,www.wikimedia.org,95.0,362.0,93.0,Standard query response 0xbe38 AAAA www.wikime...


In [588]:
df_final_resolver_target2 = df_final_resolver_target.rename({'Time_x': 'Time Target to Resolver',
                                                             'Time_y': 'Time Resolver to Target',
                                                    'Length_x': 'Length Target to Resolver',
                                                    'Length_y': 'Length Resolver to Target',
                                                    'Source_x' : 'Target in Target-Resolver',
                                                    'Destination_x' : 'Resolver in Target-Resolver',
                                                    'No._x' : 'No. in Target to Resolver' ,
                                                    'No._y' : 'No. in Resolver to Target' ,
                                                    'DNS Length_x' : 'DNS Length Target to Resolver' ,
                                                    'DNS Length_y' : 'DNS Length Resolver to Target' ,
                                                    'Time since first frame in this TCP stream_x' : 'Target to Proxy Time since first frame in this TCP stream',
                                                    'Time since first frame in this TCP stream_y' : 'Proxy to Target Time since first frame in this TCP stream',
                                                    }, axis=1)  # new method'
df_final_resolver_target2['complete query'] = list4
df_final_resolver_target2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 0 to 1276
Data columns (total 23 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   No. in Target to Resolver                                     1277 non-null   int64  
 1   Time Target to Resolver                                       1277 non-null   float64
 2   Target in Target-Resolver                                     1277 non-null   object 
 3   Resolver in Target-Resolver                                   1277 non-null   object 
 4   Protocol_x                                                    1277 non-null   object 
 5   Target to Resolver Time since first frame in this TCP stream  1277 non-null   float64
 6   Name_x                                                        1277 non-null   object 
 7   TCP Segment Len_x                                             1277 no

In [589]:
# df_merged = df_resolver_updated2.append(df_final, ignore_index=True)
df_marged_proxy_target_resolver = pd.merge(df_final, df_final_resolver_target2, on='complete query', how='outer')
df_marged_proxy_target_resolver.head()

,No. Proxy to Target,Proxy to Target Time since previous frame in this TCP stream,Source_x,Destination_x,Protocol_x_x,Proxy to Target Encrypted Query,Content-Type_x,HTTP Response in frame,Proxy to Target Time since first frame in this TCP stream,Proxy to Target HTTP-Content-Length,...,Time Resolver to Target,Source_y_y,Destination_y_y,Protocol_y_y,Resolver to Target Time since first frame in this TCP stream,Name_y,TCP Segment Len_y_y,DNS Request In,DNS Length Resolver to Target,Info_y
0,9,0.000029,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,14.0,0.000098,173.0,...,0.011421,9.9.9.9,204.102.244.58,DNS,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,245.0,122.0,Standard query response 0x62ed A client-mo.tes...
1,9,0.000029,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,14.0,0.000098,173.0,...,0.022652,8.8.8.8,204.102.244.58,DNS,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,301.0,122.0,Standard query response 0x62ed A client-mo.tes...
2,16,4.233926,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,18.0,4.610582,173.0,...,0.011421,9.9.9.9,204.102.244.58,DNS,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,245.0,122.0,Standard query response 0x62ed A client-mo.tes...
3,16,4.233926,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,18.0,4.610582,173.0,...,0.022652,8.8.8.8,204.102.244.58,DNS,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,301.0,122.0,Standard query response 0x62ed A client-mo.tes...
4,20,0.103085,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,22.0,4.828271,173.0,...,0.100845,1.1.1.1,204.102.244.58,DNS,0.176711,client-mo.test3.ch-geni-net.instageni.rnet.mis...,161.0,318.0,159.0,Standard query response 0x650a AAAA client-mo....


In [590]:
# print("this is for geni and we don't select it: ", df_final.iloc[i]['Name_y'])
df_marged_proxy_target_resolver.columns

Index(['No. Proxy to Target',
       'Proxy to Target Time since previous frame in this TCP stream',
       'Source_x', 'Destination_x', 'Protocol_x_x',
       'Proxy to Target Encrypted Query', 'Content-Type_x',
       'HTTP Response in frame',
       'Proxy to Target Time since first frame in this TCP stream',
       'Proxy to Target HTTP-Content-Length', 'TCP Segment Len_x_x',
       'Direction_x', 'No. Target to Proxy',
       'Target to Proxy - Time since previous frame in this TCP stream',
       'Source_y_x', 'Destination_y_x', 'Protocol_y_x', 'complete query',
       'Content-Type_y', 'HTTP Request in frame',
       'Target to Proxy Time since first frame in this TCP stream',
       'Target to Proxy HTTP-Content-Length', 'TCP Segment Len_y_x',
       'Time since HTTP request', 'Domain', 'Direction_y',
       'No. in Target to Resolver', 'Time Target to Resolver',
       'Target in Target-Resolver', 'Resolver in Target-Resolver',
       'Protocol_x_y',
       'Target to Resolver

In [591]:
# CHECK DATAFRAME FOR IT 
# droping if necessary
# df_marged_proxy_target_resolver = df_marged_proxy_target_resolver.drop(index=[0,1])
# df_marged_proxy_target_resolver = df_marged_proxy_target_resolver.reset_index(drop=True)

for i in range(len(df_marged_proxy_target_resolver)):
  if("client-wisconsin" in df_marged_proxy_target_resolver.iloc[i]['Name_y']):
    print("this is for geni and we don't select it: ", df_marged_proxy_target_resolver.iloc[i]['Name_y'])
    df_marged_proxy_target_resolver.drop(index=[i])

In [592]:
df_marged_proxy_target_resolver.head()


,No. Proxy to Target,Proxy to Target Time since previous frame in this TCP stream,Source_x,Destination_x,Protocol_x_x,Proxy to Target Encrypted Query,Content-Type_x,HTTP Response in frame,Proxy to Target Time since first frame in this TCP stream,Proxy to Target HTTP-Content-Length,...,Time Resolver to Target,Source_y_y,Destination_y_y,Protocol_y_y,Resolver to Target Time since first frame in this TCP stream,Name_y,TCP Segment Len_y_y,DNS Request In,DNS Length Resolver to Target,Info_y
0,9,0.000029,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,14.0,0.000098,173.0,...,0.011421,9.9.9.9,204.102.244.58,DNS,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,245.0,122.0,Standard query response 0x62ed A client-mo.tes...
1,9,0.000029,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,14.0,0.000098,173.0,...,0.022652,8.8.8.8,204.102.244.58,DNS,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,301.0,122.0,Standard query response 0x62ed A client-mo.tes...
2,16,4.233926,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,18.0,4.610582,173.0,...,0.011421,9.9.9.9,204.102.244.58,DNS,0.373081,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,245.0,122.0,Standard query response 0x62ed A client-mo.tes...
3,16,4.233926,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,18.0,4.610582,173.0,...,0.022652,8.8.8.8,204.102.244.58,DNS,0.110833,client-mo.test3.ch-geni-net.instageni.rnet.mis...,124.0,301.0,122.0,Standard query response 0x62ed A client-mo.tes...
4,20,0.103085,192.122.236.103,204.102.244.58,HTTP,Go-http-client/1.1,application/oblivious-dns-message,22.0,4.828271,173.0,...,0.100845,1.1.1.1,204.102.244.58,DNS,0.176711,client-mo.test3.ch-geni-net.instageni.rnet.mis...,161.0,318.0,159.0,Standard query response 0x650a AAAA client-mo....


In [593]:
# saving the dataframe
df_marged_proxy_target_resolver.to_csv('final2.csv')